In [1]:
import pandas as pd
import re
import datetime
import requests
import bs4
import utils.craigslist as cl

In [2]:
pd.set_option("display.max_rows", 500)

## Scrap main pages

In [3]:
craigslist = cl.scraper("https://berlin.craigslist.org/d/housing/search/hhh")

In [5]:
craigslist

,id,url,date,description,price,hood,housing,img_url
0,7075722295,https://berlin.craigslist.org/apa/d/exceptiona...,2020-02-15 14:52,Exceptional 2 Bedroom Maisonette Rooftop Views,€800,Mitte,2br - 112m,https://images.craigslist.org/00404_hvNQ0G2OSe...
1,7075722186,https://berlin.craigslist.org/apa/d/exceptiona...,2020-02-15 14:52,Exceptional 2 Bedroom Maisonette Rooftop Views,€800,Mitte,2br - 112m,https://images.craigslist.org/00404_hvNQ0G2OSe...
2,7075722094,https://berlin.craigslist.org/apa/d/exceptiona...,2020-02-15 14:51,Exceptional 2 Bedroom Maisonette Rooftop Views,€800,Mitte,2br - 112m,https://images.craigslist.org/00404_hvNQ0G2OSe...
3,7075721993,https://berlin.craigslist.org/apa/d/exceptiona...,2020-02-15 14:51,Exceptional 2 Bedroom Maisonette Rooftop Views,€800,Mitte,2br - 112m,https://images.craigslist.org/00404_hvNQ0G2OSe...
4,7075682980,https://berlin.craigslist.org/roo/d/sunny-room...,2020-02-15 12:15,SUNNY rOOM wITH sINGLE bED fOR rENT iN sCHÖNEBERG,€499,Willmanndamm,,https://images.craigslist.org/00808_7eQd9y0K3m...
...,...,...,...,...,...,...,...,...
706,7034471231,https://berlin.craigslist.org/vac/d/berln-piso...,2020-01-01 16:21,BERLÍN PISO VACACIONES 1 HABITACIÓN CENTRO PRE...,€50,BERLIN,1br - 45m,https://images.craigslist.org/00l0l_gGSgMQGGEE...
707,7034474478,https://berlin.craigslist.org/vac/d/berln-piso...,2020-01-01 16:21,BERLÍN PISO PARA LAS VACACIONES 2 HABITACIÓNES...,€60,BERLIN,2br - 50m,https://images.craigslist.org/00J0J_dHfwjJegEx...
708,7034462358,https://berlin.craigslist.org/vac/d/berlin-fer...,2020-01-01 16:21,BERLIN FERIENWOHNUNG ZENTRAL 2 ZIMMER UNTERKUN...,€60,BERLIN,2br - 50m,https://images.craigslist.org/00B0B_9e9QesZEU6...
709,7034477673,https://berlin.craigslist.org/vac/d/berln-piso...,2020-01-01 16:20,BERLÍN PISO PARA LAS VACACIONES 2 HABITACIÓNES...,€70,,2br - 64m,https://images.craigslist.org/00Z0Z_lBAbHpWNjG...


### Save dataframe to JSON file oriented to table

In [6]:
craigslist.to_json("data/craigslist.json", orient="table")